<a href="https://colab.research.google.com/github/fernandodeeke/can2025/blob/main/eliminacaoGaussiana2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center><h1> <h2></h2></h1></center>
<center><h1>Análise Numérica</h1></center>
<center><h2>2025/1</h2></center>
<center><h3>Fernando Deeke Sasse</h3></center>
<center><h3>CCT - UDESC</h3></center>
<center><h2>Eliminação de Gauss - 2</h2></center>

### 1. Uma visão diferente do método de eliminação de Gauss

Como vimos anteriormente, o método de eliminação de Gauss é o método direto mais amplamente utilizado para resolver sistemas lineares e é a base para todas as variantes dos métodos de eliminação. Vamos agora revisitar os dois passos básicos:

1. **Fase de eliminação**: realizamos a eliminação dos elementos abaixo da diagonal principal da matriz aumentada $[A|B]$ por meio de operações elementares nas linhas, até obtermos a forma escalonada.
2. **Substituição retroativa**: resolvemos recursivamente as equações resultantes de trás para frente.

Descrevemos cada um desses passos abaixo, agora utilizando fatores de eliminação, mais adequados para a implementação algorítmica do método.

### 2. Fase de eliminação
Chamamos de *linha pivô* a linha acima da qual todos os elementos abaixo da diagonal principal já foram zerados.
Na matriz aumentada, seja a linha pivô $L_k$ e uma linha típica $L_i$ abaixo dela a ser transformada, para zerar o elemento $a_{ik}$. Para isso, devemos multiplicar a linha pivô por $\lambda = a_{ik}/a_{kk}$ e subtrair o resultado de $L_i$, ou seja,

$$
L_i \rightarrow L_i - \lambda L_k\,.
$$

Os elementos da linha $L_i$ são atualizados da seguinte forma:

\begin{align}
&a_{ij} \leftarrow a_{ij} - \lambda a_{kj}\,\qquad j=k, \ldots , n\\
&b_i \leftarrow b_i - \lambda b_k\,.
\end{align}

Como $a_{ik}=0$ por construção, para economizar tempo computacional, ele não precisa ser calculado, então podemos usar $j=k+1, \ldots , n$.

O índice $k$ é o índice da linha pivô, de modo que $k=1, \ldots, n-1$. O índice $i$ designa a linha a ser transformada, de modo que $i=k+1,\ \ldots, n$.

Lembre-se de que, ao usar range(i,j), o intervalo real é $1, 2, \ldots, j-1$. Por exemplo,

In [ ]:
import numpy as np
c=np.array([1,2,3,4])
print(c[0])
print(c[-1])

1
4


In [ ]:
for j in range(0,3):
    print(c[j])

1
2
3


### 3. Algoritmo de eliminação de Gauss

O pseudocódigo é o seguinte (supondo índices de 1 a n):

eliminacaoGaussiana(A, b)
    # A é a matriz de coeficientes (n × n)
    # b é o vetor de termos independentes (n × 1)
    n = len(b)
    for k = 1 to n-1 do          # k é a linha pivô
        for i = k+1 to n do      # i é a linha a ser eliminada
            m = A[i,k] / A[k,k]  # calcula o multiplicador
            for j = k+1 to n do  # atualiza os elementos da linha i
                A[i,j] = A[i,j] - m * A[k,j]
            end do
            A[i,k] = 0  # define o elemento abaixo do pivô como zero
            b[i] = b[i] - m * b[k]  # atualiza o termo independente
        end do
    end do
    return [A|b]  # retorna a matriz aumentada com A e b concatenados


O algoritmo de eliminação pode ser implementado em Python como mostrado abaixo. Observe que o último elemento da matriz a tem índice n-1 e que range(0,n) vai de 0 a n-1.

In [ ]:
def GaussElimin(a,b):
    n = len(a)
    for k in range(0,n): # define o índice do elemento pivô
        for i in range(k+1,n): # percorre a linha abaixo da linha pivô até n-1 (última linha)
            lam = a[i,k]/a[k,k]
            a[i,k+1:n] = a[i,k+1:n] - lam*a[k,k+1:n] # atualiza os elementos da linha i
            a[i,k]=0
            b[i] = b[i] - lam*b[k]
    return  np.hstack([a,b])

Vamos testar a função:

In [ ]:
A = np.array([[6.,1.,2.,4.,6],[5.,11.,-3.,2.,2],[-3.,4.,3.,5.,-3],[5.,2.,8.,-3,3.],[2,5,6,5,9.]])
print(A)

[[ 6.  1.  2.  4.  6.]
 [ 5. 11. -3.  2.  2.]
 [-3.  4.  3.  5. -3.]
 [ 5.  2.  8. -3.  3.]
 [ 2.  5.  6.  5.  9.]]


In [ ]:
B = np.transpose(np.array([[2.,-4.,3.,-7.,3.]]))
print(B)

[[ 2.]
 [-4.]
 [ 3.]
 [-7.]
 [ 3.]]


Vamos formar a matriz aumentada $M$:

In [ ]:
M = np.hstack([A,B])
print(M)

[[ 6.  1.  2.  4.  6.  2.]
 [ 5. 11. -3.  2.  2. -4.]
 [-3.  4.  3.  5. -3.  3.]
 [ 5.  2.  8. -3.  3. -7.]
 [ 2.  5.  6.  5.  9.  3.]]


Realize a eliminação de Gauss:

In [ ]:
M1=GaussElimin(A,B)
print(M1)

[[  6.           1.           2.           4.           6.
    2.        ]
 [  0.          10.16666667  -4.66666667  -1.33333333  -3.
   -5.66666667]
 [  0.           0.           6.06557377   7.59016393   1.32786885
    6.50819672]
 [  0.           0.           0.         -14.77567568  -3.15945946
  -15.38648649]
 [  0.           0.           0.           0.           7.82586428
    2.19901226]]


### 4. Fase de substituição retroativa

Começamos resolvendo a última equação correspondente do sistema reduzido, ou seja,

$$
x_n=\frac{b_n}{a_{nn}}\,.
$$

Consideremos agora a etapa de substituição retroativa onde $x_n, x_{n-1}, \ldots , x_{k+1} $ já foram calculados e devemos calcular $x_k$ a partir da equação $k$:

$$
a_{kk}x_k+a_{k k+1} x_{k+1}+ \cdots + a_{k n}x_n=b_k\,.
$$

Resolvendo para $x_k$, obtemos

$$
x_k=\frac{1}{a_{kk}}\left(b_k-a_{k k+1} x_{k+1}- \cdots - a_{k n}x_n\right)= \frac{1}{ a_{kk}}\left(b_k-\sum_{j=k+1}^n a_{kj}x_j\right)\,.
$$

Vamos implementar essa função em Python.

### 5. Algoritmo de substituição retroativa
A função a seguir toma como entrada uma matriz de coeficientes na forma escalonada e a matriz-coluna da parte homogênea. A saída é a solução do sistema linear correspondente.

In [ ]:
def GaussRetro(a, b):
    n = len(b)
    x = np.zeros(n)
    x[n-1] = (b[n-1] / a[n-1, n-1]).item()
    for k in range(n-2, -1, -1):
        x[k] = ((b[k] - np.dot(a[k, k+1:n], x[k+1:n])) / a[k, k]).item()
    return np.transpose([x])

Para entender o uso do comando np.dot acima, devemos notar que, para cada $k$, tanto $a[k,k+1:n]$ quanto $x[k+1:n]$ são vetores computacionais de comprimento $n-k$. O comando np.dot faz com que os componentes respectivos sejam multiplicados e somados, como no produto escalar usual. O comando range(n-1,-1,-1) faz com que os valores de $k$ comecem em $n-1$ e diminuam até chegar a $k=0$ (que precede -1). O método .item() extrai o valor escalar da quantidade.

Vamos usar o exemplo acima. Inicialmente, dividimos a matriz aumentada. A matriz de coeficientes é dada por

In [ ]:
A1 = M1[:,0:5]
A1

array([[  6.        ,   1.        ,   2.        ,   4.        ,
          6.        ],
       [  0.        ,  10.16666667,  -4.66666667,  -1.33333333,
         -3.        ],
       [  0.        ,   0.        ,   6.06557377,   7.59016393,
          1.32786885],
       [  0.        ,   0.        ,   0.        , -14.77567568,
         -3.15945946],
       [  0.        ,   0.        ,   0.        ,   0.        ,
          7.82586428]])

A parte não homogênea é dada por:

In [ ]:
B1= M1[:,5]
B1

array([  2.        ,  -5.66666667,   6.50819672, -15.38648649,
         2.19901226])

In [ ]:
B1v = np.reshape(B1,(5,1))

Vamos agora aplicar a função que realiza a substituição retroativa:

In [ ]:
X1 = GaussRetro(A1,B1v)
X1

array([[-0.45549738],
       [-0.44511967],
       [-0.21643605],
       [ 0.98125467],
       [ 0.28099289]])

Vamos verificar o resultado:

In [ ]:
A@X1-B

array([[ 0.0000000e+00],
       [-8.8817842e-16],
       [ 0.0000000e+00],
       [ 0.0000000e+00],
       [ 0.0000000e+00]])

Podemos resolver sistemas lineares com métodos diretos usando o comando solve do numpy:

In [ ]:
import numpy.linalg as la

In [ ]:
x = la.solve(A,B)
print(x)

[[-0.45549738]
 [-0.44511967]
 [-0.21643605]
 [ 0.98125467]
 [ 0.28099289]]


### 6. Algoritmo completo de eliminação de Gauss

Vamos combinar as duas funções em uma única. A atribuição a[i,k]=0 agora pode ser removida porque é irrelevante:

In [ ]:
import numpy as np

In [ ]:
def GaussSolve(a, b):
    n = len(a)
    a = a.copy()  # Trabalha com uma cópia para preservar a matriz original
    b = b.copy()  # Trabalha com uma cópia para preservar o vetor original
    x = np.zeros(n)
    for k in range(0, n-1):
        for i in range(k+1, n):
            lam = a[i, k] / a[k, k]
            a[i, k+1:n] = a[i, k+1:n] - lam * a[k, k+1:n]
            b[i] = b[i] - lam * b[k]
    x[n-1] = b[n-1] / a[n-1, n-1]
    for k in range(n-2, -1, -1):
        x[k] = (b[k] - np.dot(a[k, k+1:n], x[k+1:n])) / a[k, k]
    return x

Vamos testar o algoritmo completo no sistema linear anterior.

In [ ]:
A = np.array([[6., 1., 2., 4.], [5., 11., -3., 2.], [-3., 4., 3., 5.], [5., 2., 8., 3.]])
B = np.array([2., -4., 3., -7.])

In [ ]:
X =  GaussSolve(A,B)
X

array([-0.32152756, -0.84200801, -1.1210348 ,  1.75331075])

Vamos verificar:

In [ ]:
residual = A @ X - B  # Calcula o resíduo
residual

array([ 0.00000000e+00, -8.88178420e-16,  8.88178420e-16, -3.55271368e-15])

In [ ]:
norm_inf = np.linalg.norm(residual, ord=np.inf)
print("\nNorma Infinita do Resíduo:")
print(norm_inf)


Norma Infinita do Resíduo:
3.552713678800501e-15


### 7. Exemplo com sistemas grandes

Vamos testar sistemas aleatórios maiores

In [ ]:
np.random.seed(43453)
N = 10
A3 = np.random.rand(N,N)
B3 = np.random.rand(N)
B3a = B3.reshape(N,1)

Note que

In [ ]:
B3

array([0.79296977, 0.19843862, 0.88683062, 0.02492198, 0.43015755,
       0.19245975, 0.47326455, 0.0801523 , 0.45018061, 0.84562416])

In [ ]:
B3a

array([[0.79296977],
       [0.19843862],
       [0.88683062],
       [0.02492198],
       [0.43015755],
       [0.19245975],
       [0.47326455],
       [0.0801523 ],
       [0.45018061],
       [0.84562416]])

In [ ]:
X3 = GaussSolve(A3,B3)
X3

array([[-0.78014194],
       [ 1.64411949],
       [-1.13319994],
       [-0.72833755],
       [-2.02054919],
       [ 1.06249775],
       [-0.78987105],
       [ 2.26036594],
       [ 2.07989874],
       [-0.4584725 ]])

In [ ]:
np.random.seed(43453)
N = 10
A3 = np.random.rand(N,N)

Vamos verificar a resposta calculando o resíduo:

In [ ]:
R = A3@X3-B3a
R

array([[-4.44089210e-16],
       [-3.33066907e-16],
       [-5.10702591e-15],
       [-7.32747196e-15],
       [-2.22044605e-15],
       [ 2.10942375e-15],
       [-4.44089210e-16],
       [-3.21964677e-15],
       [-1.99840144e-15],
       [ 7.77156117e-16]])

Se o sistema for muito grande, é difícil inspecionar todos os componentes. Em geral, avaliamos o resíduo calculando uma norma do vetor correspondente. A norma mais comumente usada é a norma infinita, definida como sendo a maior magnitude entre os componentes:

$$
|\mathbf{u}|_{\infty}=\max_i\{{|u_i|},i=1,\ldots,n\}.
$$

No nosso caso,

In [ ]:
import numpy.linalg as la

In [ ]:
NR = la.norm(R, np.inf)
NR

7.327471962526033e-15

Vamos testar o desempenho do programa que escrevemos com sistemas grandes.

In [ ]:
np.random.seed(43453)
N = 300
A4 = np.random.rand(N,N)
B4 = np.random.rand(N)

In [ ]:
X4 = GaussSolve(A4,B4)

Vamos calcular o resíduo:

In [ ]:
np.random.seed(43453)
N = 300
A4 = np.random.rand(N,N)
B4 = np.random.rand(N,1)

In [ ]:
NR = la.norm(A4@X4-B4,np.inf)
NR

6.478706460200101e-12

Vamos estimar o tempo de CPU necessário para resolver este sistema:

In [ ]:
np.random.seed(43453)
N = 300
A4 = np.random.rand(N,N)
B4 = np.random.rand(N)

In [ ]:
%%timeit
GaussSolve(A4,B4)

249 ms ± 46.4 ms por loop (média ± desv. padrão de 7 execuções, 1 loop cada)


O tempo de CPU necessário usando a função solve do numpy é dado por:

In [ ]:
np.random.seed(43453)
N = 300
A4 = np.random.rand(N,N)
B4 = np.random.rand(N)

In [ ]:
%%timeit
la.solve(A,B)

6.3 µs ± 1.65 µs por loop (média ± desv. padrão de 7 execuções, 100000 loops cada)


Como esperado, o solver é muito mais rápido que nosso algoritmo pedagógico, por quase quatro ordens de grandeza.

### 8. Exercícios

**1.** Resolva passo a passo um sistema linear aleatório $4 \times 4$ (use os primeiros 4 dígitos do seu CPF como semente), descrevendo cada etapa dos algoritmos GaussElimin e GaussRetro. Verifique a resposta calculando o resíduo.

**2.** Resolva um sistema aleatório $500 \times 500$ usando o algoritmo GaussSolve (não mostre o resultado). Verifique a correção do resultado calculando a norma do resíduo e compare o tempo de CPU necessário. Repita o procedimento usando o solver do numpy.

**3.** O **número de condição** de uma matriz $ A $, denotado por $ \kappa(A) $, mede a sensibilidade da solução de um sistema linear $ A \mathbf{x} = \mathbf{b} $ a pequenas perturbações na matriz $ A $ ou no vetor $ \mathbf{b} $. Ele é definido como:
$$
\kappa(A) = \|A\| \cdot \|A^{-1}\|,
$$
onde $ \|A\| $ é a norma da matriz (geralmente a norma 2). Um número de condição alto indica que o sistema é mal condicionado, ou seja, pequenas mudanças nos dados podem causar grandes variações na solução. Um exemplo de matriz mal condicionada é a matriz de Hilbert $H_n$ de ordem $n$, definida por
$$
H_{ij} = \frac{1}{i + j + 1}, \quad i, j = 0, 1, \dots, n.
$$

Podemos definir em Python como segue:

In [ ]:
def hilb(n):
    return np.array([[1/(i+j+1) for i in range(n)] for j in range(n)])
    return np.array([[1/(i+j+1) for i in range(n)] for j in range(n)])

Por exemplo,

In [ ]:
hilb(4)

array([[1.        , 0.5       , 0.33333333, 0.25      ],
       [0.5       , 0.33333333, 0.25      , 0.2       ],
       [0.33333333, 0.25      , 0.2       , 0.16666667],
       [0.25      , 0.2       , 0.16666667, 0.14285714]])

(i) Resolva o sistema linear $HX=B$, onde $H$ é a matriz de Hilbert $H(30)$, $30 \times 30$, e $B$ é uma matriz-coluna com todos os elementos iguais a 1. Use o algoritmo de eliminação de Gauss desenvolvido anteriormente para resolver o sistema. Calcule o resíduo em ambos os casos.

(ii) Compare o número de condição de $H$ com o de uma matriz aleatória $30 \times 30$.

(iii) Mostre que uma pequena perturbação em um componente de $H(30)$ faz com que a solução do sistema definido em (i) mude drasticamente.


**4.** Estude a instabilidade de sistemas com matrizes de Hilbert $n \times n$. Por exemplo, varie ligeiramente o valor de um coeficiente de $B$, ou adicione um pequeno termo à matriz $H$. Use o solver do Numpy.

**5.** Defina um sistema $GX=B$, $n \times n$, com $G=[g_{ij}]$ e $g_{ij}=1/(1+i+2j)$. Verifique se tal sistema também é instável. Varie $n$. Use o solver do Numpy.

In [ ]:
import numpy as np
def m5(n):
    return np.array([[1/(i+2*j+1) for i in range(n)] for j in range(n)])

In [ ]:
m5(5)

array([[1.        , 0.5       , 0.33333333, 0.25      , 0.2       ],
       [0.33333333, 0.25      , 0.2       , 0.16666667, 0.14285714],
       [0.2       , 0.16666667, 0.14285714, 0.125     , 0.11111111],
       [0.14285714, 0.125     , 0.11111111, 0.1       , 0.09090909],
       [0.11111111, 0.1       , 0.09090909, 0.08333333, 0.07692308]])